In [1]:
from pinntorch import *
from functools import partial
from pinntorch.LBFGS import FullBatchLBFGS

In [2]:
K = 5.0

In [3]:
def exact_solution_log(x):
    return 1/(1+torch.exp(-torch.Tensor(K*x)))

# exact solution in NumPy: This one is needed for the loss function becasue somehow the tensor form does not work as of now.
def exact_solution_log_np(x):
    return 1/(1+np.exp(-K*x))

def create_noisy_data(x, std_dev, noise_seed = 42):
    exact = exact_solution_log(x)
    torch.manual_seed(noise_seed)

    return exact + torch.randn(exact.size())*std_dev 

def data_loss(model: PINN, data: torch.Tensor = None, x: torch.Tensor = None) -> torch.float:
    # MSE loss 
    return (f(model, x) - data).pow(2).mean()

def physics_loss(model: PINN, x: torch.Tensor = None) -> torch.float:
    # define PDE loss
    #x = generate_sample(20, (-1.0, 1.0))
    pde_loss_pre = df(model, x) - K*f(model, x)*(1 - f(model, x))
    pde_loss = pde_loss_pre.pow(2).mean()
    
    # define conditional losses (initial + boundary)
    boundary_loss_right_pre = (f(model, at(+1.0)) - exact_solution_log_np(+1)) 
    boundary_loss_right = boundary_loss_right_pre.pow(2).mean()

    # combine all losses
    final_loss = pde_loss + boundary_loss_right
    
    return final_loss

def generate_random_mask(size, num_true=5):
    mask = torch.zeros(size, dtype=torch.bool)
    indices = torch.randperm(size)[:num_true]
    mask[indices] = True
    return mask

def total_loss(model: PINN, data: torch.Tensor = None, x: torch.Tensor= None) -> list:

    """adds the physics and the data loss with coefficients alpha and (1-alpha) respectively"""
    #mask = generate_random_mask(20, 5)
    #masked_data = data[mask]
    #masked_x = x[mask]

    loss_data = data_loss(model, data, x)

    loss_physics = physics_loss(model, x)

    return loss_data, loss_physics

In [4]:
class ValLRMonitor(EpochCallBack):
    """
    Abstract base class for epoch callback objects.
    """
    def __init__(self, validation_points):
        self.val_points = validation_points
        self.val_loss_fn = partial(physics_loss, x=self.val_points)

    def prepare(self, max_epochs, model, loss_fn, optimizer):
        self.val_history = []
        self.lr_history = []

    def process(self, epoch, model, loss_fn, optimizer, current_loss, extra_logs):
        loss_val = self.val_loss_fn(model)
        loss_physics_val = loss_val.detach().numpy()
        self.lr_history.append(float(optimizer.param_groups[0]["lr"]))
        self.val_history.append(loss_physics_val)

In [5]:
def population_training(settings, input_data, train_points, val_points):

    L_p = []
    L_d = []
    L_VAL = []
    LR = []
    
    models_trained = []

    torch.manual_seed(settings['model_seed'])
    for i in range(settings['population_size']):
        print(i)
        loss_fn = partial(total_loss,data = input_data, x=train_points)  # For each alpha we need a loss function with different alpha. 
             
        model = PINN(1, 3, 9, 1)
        
        mgda = WeightMethods(
            method=getattr(Moo_method, "mgda"),
            n_tasks=2,
            # normalization=config.moo_normalization, # mgda
            device=device,
        )

        callbacks = [TrainLossMonitor(), ValLRMonitor(val_points)]
        trained_model = train_model(
            model = model, 
            loss_fn=loss_fn,
            #mo_method=mgda,
            max_epochs = settings['epochs'],
            lr_decay=1e-3,
            optimizer_fn = partial(torch.optim.SGD, lr=settings['start_learning_rate']),
            epoch_callbacks = callbacks
        )


        L_p.append(np.array(callbacks[0].loss_history[1]))
        L_d.append(np.array(callbacks[0].loss_history[0]))
        LR.append(np.array(callbacks[1].lr_history))
        L_VAL.append(np.array(callbacks[1].val_history))
        models_trained.append(trained_model)

    return L_p, L_d, LR, L_VAL, models_trained

In [6]:
settings = {}
alphas = 1-torch.logspace(start=-2, end=0.0, steps=20, base=80)

settings['n_data_points'] = 20
settings['n_train_points'] = 20
settings['n_val_points'] = 39
settings['noise_level'] = 0.1
settings['epochs'] = 3
settings['population_size'] = 1
settings['noise_seed'] = 123
settings['model_seed'] = 333

training_points = generate_grid((settings['n_train_points']), (-1.0,1.0))
validation_points = generate_grid((settings['n_val_points']), (-1.0,1.0))

data_noise = create_noisy_data(training_points, settings['noise_level'], noise_seed=settings['noise_seed'])

settings['start_learning_rate'] = 0.003
learning_rate = settings['start_learning_rate']
epochs = settings['epochs']

def custom_color_normalize(value):
    return value**80

#until = [9342, 5509, 5686, 6044, 6874, 8023, 9543, 8622, 2639, 2799, 3227, 3876, 128, 594, 169, 253, 410, 521, 2051, 49999]

Loss_physics, Loss_data, LR_evolution, Loss_val, models_trained = population_training(settings, data_noise, training_points, validation_points)

0
tensor([ 0.1931, -0.0531,  0.2715, -0.1283,  0.4312, -0.0948, -0.7740,  0.2923,
        -0.7242,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.4061, -0.5756, -0.2513, -0.1058,  0.7556, -0.6233,
         0.3206,  0.6702,  0.2060,  0.0000,  0.2955, -0.0567, -0.1058,  0.5301,
         0.2712, -0.1415,  0.0505, -0.4882,  0.0104,  0.5770,  0.4781,  0.3218,
         0.2379, -0.4256,  0.2317,  0.4046, -0.2043, -0.0849,  0.2614,  0.3225,
        -0.0299,  0.1104,  0.4705, -0.1131, -0.4947, -0.1415, -0.0039, -0.5565,
         0.2833,  0.3858,  0.4362,  0.3946,  0.2032, -0.2013,  0.3049, -0.3904,
        -0.1187, -0.3677, -0.0955,  0.4539, -0.2595, -0.4652, -0.1490, -0.1232,
        -0.2446, -0.3536, -0.4240, -0.4896, -0.3478, -0.2339,  0.0349,  0.3865,
        -0.1071, -0.5377,  0.2709,  0.0415,  0.0983,  0.1459, -0.4790,  0.4550,
        -0.5112, -0.2801, -0.5719,  0.4733, -0.5769, -0.0804,  0.3464, -0.5504,
         0.4967, -0.3262,  0.5173,  0.

c:\Users\Admin\Repositories\MoPINN\pinntorch\_training.py:130: RuntimeWarning: No multi-objective optimization method (mo_method) was set, despite defining multiple losses. Training will continue with the sum of the losses!
  warnings.warn("No multi-objective optimization method (mo_method) was set, despite defining multiple losses. Training will continue with the sum of the losses!", RuntimeWarning)


In [21]:
run_name = 'mgda_lbfgs'

result_dict = {
    "settings" : settings,
    "input_data": data_noise.detach().cpu().numpy(),
    "loss_data": Loss_data,
    "loss_physics": Loss_physics,
    "LR": LR_evolution,
    "loss_val": Loss_val
}
    
path = create_run_folder(run_name)
save_dictionary(path, run_name, result_dict)
save_models(path, models_trained)

#print(data_noise.shape)
#save_results(run_name, settings, data_noise, Loss_data, Loss_physics, LR_evolution, Loss_val, models_trained)


settings
input_data
loss_data
loss_physics
LR
loss_val
